<a href="https://colab.research.google.com/github/izzZk-hub/limpieza_datos_y_eda/blob/main/3_ejercicios_pandas_delitos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Análisis exploratorio de incidencia delictiva en México

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
df = pd.read_csv('data/delitos.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'data/delitos.csv'

#### Ejercicio 1: Elige 3 estados de la república y grafica una serie de tiempo de la frecuencia abosluta de homicidios dolosos de enero 2015 a julio 2019 en estas tres entidades

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv('./data/datos_delitos.csv', encoding='latin1')

estados = ['Ciudad de México', 'Jalisco', 'Nuevo León']

df_hom = df[df['Delito'] == 'HOMICIDIO DOLOSO']
df_hom_sel = df_hom[df_hom['Entidad'].isin(estados)]

df_hom_sel['Fecha'] = pd.to_datetime(df_hom_sel['Fecha'], format='%Y-%m-%d')

serie_tiempo = df_hom_sel.groupby(['Fecha', 'Entidad'])['Frecuencia'].sum().unstack()

serie_tiempo.plot(figsize=(12,6), marker='o')
plt.title('Homicidios dolosos (Ene 2015 - Jul 2019) en 3 estados')
plt.ylabel('Frecuencia absoluta')
plt.xlabel('Fecha')
plt.grid(True)
plt.show()

#### Ejercicio 2: Contetas las siguientes  preguntas:
1. ¿Cuántos homicidios dolosos hubo en Colima en el 2018?
2. ¿Cuantos robos de vehículo automotor ha habido en el 2019?
3. Obten la suma de homicidos dolosos y feminidios en toda la República Mexicana en cada año.
4. ¿En qué mes y en qué municipio ha ocurrido el mayor número de feminicidios?
5. ¿En qué año y en qué estado ha ocurrido el mayor número de feminicidios?

In [ ]:
import pandas as pd

df = pd.read_csv('./data/datos_delitos.csv', encoding='latin1')

colima_2018 = df[(df['Entidad'] == 'Colima') & (df['Delito'] == 'HOMICIDIO DOLOSO') & (df['Año'] == 2018)]['Frecuencia'].sum()
print(f"1. Homicidios dolosos en Colima en 2018: {colima_2018}")

robos_2019 = df[(df['Delito'] == 'ROBO DE VEHÍCULO AUTOMOTOR') & (df['Año'] == 2019)]['Frecuencia'].sum()
print(f"2. Robos de vehículo automotor en 2019: {robos_2019}")

suma_hom_fem = df[df['Delito'].isin(['HOMICIDIO DOLOSO','FEMINICIDIO'])].groupby('Año')['Frecuencia'].sum()
print("3. Suma de homicidios dolosos y feminicidios por año en toda la República Mexicana:")
print(suma_hom_fem)

fem_max = df[df['Delito'] == 'FEMINICIDIO'].loc[df[df['Delito'] == 'FEMINICIDIO']['Frecuencia'].idxmax()]
print(f"4. Mes y municipio con el mayor número de feminicidios: {fem_max['Mes']}, {fem_max['Municipio']}")

fem_max_estado = df[df['Delito'] == 'FEMINICIDIO'].loc[df[df['Delito'] == 'FEMINICIDIO']['Frecuencia'].idxmax()]
print(f"5. Año y estado con el mayor número de feminicidios: {fem_max_estado['Año']}, {fem_max_estado['Entidad']}")

#### Ejercicio 3: Haz una gráfica de pastel de tipos de delito. Deberás crear una gráfica para cada año. Utilzia la función subplots de matplotlib

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

df = pd.read_csv('./data/datos_delitos.csv', encoding='latin1')

años = sorted(df['Año'].unique())
fig, axes = plt.subplots(nrows=len(años)//3 + 1, ncols=3, figsize=(15, 5*len(años)//3))
axes = axes.flatten()

for i, año in enumerate(años):
    df_año = df[df['Año'] == año].groupby('Delito')['Frecuencia'].sum()
    axes[i].pie(df_año, labels=df_año.index, autopct='%1.1f%%', startangle=90)
    axes[i].set_title(f'Tipos de delito en {año}')

for j in range(i+1, len(axes)):
    axes[j].axis('off')

plt.tight_layout()
plt.show()

---
#### Calcula la tasa por 100,000 habitantes
##### Tasa por 100,000 habitantes
Mostrar el total de delitos en una entidad no nos sirve de mucho. Es mucho más útil calcular la tasa de incidencia delictiva por cada 100,000 habitantes

$$
tasa = \frac{delitos\space totales}{población} \times 100,000
$$

Esta tasa la podemos anualizar multiplicándola por un factor de 12
$$
tasa\space anualizada = tasa \times 12
$$

Población por entidad federativa según [la encuesta intercensal 2015](https://www.inegi.org.mx/programas/intercensal/2015/)

No tienes que descargar nada. Ya están los datos en la carpeta data

In [ ]:
pobs = pd.read_csv('data/poblacion_entidades_2015.csv', encoding='iso-8859-1', sep=";")
pobs = pobs[['Cve_Entidad', 'Entidad', 'Poblacion']]
pobs = pobs.rename(columns={'Cve_Entidad':'clave_entidad', 'Entidad':'entidad', 'Poblacion':'poblacion'})
pobs.head()

In [ ]:
import pandas as pd

df = pd.read_csv('./data/datos_delitos.csv', encoding='latin1')
pobs = pd.read_csv('data/poblacion_entidades_2015.csv', encoding='iso-8859-1', sep=";")
pobs = pobs[['Cve_Entidad', 'Entidad', 'Poblacion']]
pobs = pobs.rename(columns={'Cve_Entidad':'clave_entidad', 'Entidad':'entidad', 'Poblacion':'poblacion'})

df = df.merge(pobs, left_on='Entidad', right_on='entidad', how='left')
df['tasa'] = (df['Frecuencia'] / df['poblacion']) * 100000
df['tasa_anualizada'] = df['tasa'] * 12

df.head()